In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS fleetops_project.silver;

CREATE TABLE IF NOT EXISTS fleetops_project.silver.iot_telemetry (
    event_id STRING,
    truck_id INT,
    event_time TIMESTAMP,
    engine_rpm INT,
    speed_kmh INT,
    fuel_level INT,
    gps_latitude DOUBLE,
    gps_longitude DOUBLE,
    engine_temp INT,
    _ingested_at TIMESTAMP
  ) USING DELTA
  PARTITIONED BY (event_date DATE);

In [0]:
%sql

CREATE OR REPLACE TEMP VIEW vw_telemetry_clean AS
SELECT
    event_id,
    CAST(truck_id AS INT) AS truck_id,
    CAST(timestamp AS TIMESTAMP) AS event_time,
    CAST(engine_rpm AS INT) AS engine_rpm,
    CAST(speed_kmh AS INT) AS speed_kmh,
    CAST(fuel_level AS INT) AS fuel_level, 
    CAST(gps_latitude AS DOUBLE) AS gps_latitude,
    CAST(gps_longitude AS DOUBLE) AS gps_longitude,
    CAST(engine_temp AS INT) AS engine_temp,
    CAST(timestamp AS DATE) as event_date,
    _ingested_at
FROM fleetops_project.bronze.iot_telemetry
WHERE timestamp IS NOT NULL;

CREATE OR REPLACE TEMP VIEW vw_telemetry_dedup AS
SELECT * EXCEPT(rn)
FROM (
    SELECT 
        *,
        ROW_NUMBER() OVER (
            PARTITION BY event_id
            ORDER BY _ingested_at DESC
        ) AS rn
    FROM vw_telemetry_clean
)
WHERE rn = 1;

MERGE WITH SCHEMA EVOLUTION INTO fleetops_project.silver.iot_telemetry t
USING vw_telemetry_dedup s
ON t.event_id = s.event_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;